$\newcommand{\coeff}{\frac{\lVert C \rVert_\infty}{\epsilon}}$

In [89]:
from clustering import entropic_wasserstein_sin, entropic_wasserstein_sin_warm
from clustering.distribution import ContinuousDistribution, FiniteDistribution
from clustering.utils.plot_utils import add_linear_regression

import numpy as np
import pandas as pd 
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


import itertools

In [2]:
from scipy.spatial.distance import cdist
from ot import emd2

def wasserstein(X, Y):
    cost = cdist(X, Y, metric="sqeuclidean")
    return emd2(np.ones(X.shape[0])/X.shape[0], np.ones(Y.shape[0])/Y.shape[0], cost)

def lineplot(x, var, data, y='value', ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    sns.lineplot(x, y, data=data[data['var'] == var], ax=ax, **kwargs)
    ax.set_ylabel(var)
    ax.set_xlabel(x)
    return ax

## Number of iterations vs. $\coeff$

### Setup

$100$ points are drawn from a mixture of 5 Gaussian whose mean is in $[0, 1]$. The number of iterations needed to reach a precision of $\epsilon$ is computed. This process is repeated over $50$ runs. 

In [68]:
n_points = 100
n_batch = 50
eps_to_try = np.logspace(1.2, -.5, 10)
tau_to_try = np.logspace(-1, -4, 4)

result = pd.DataFrame(columns=pd.Index(range(n_batch), name="run"),
                      index=pd.MultiIndex.from_product([eps_to_try, tau_to_try,
                                                        ['n_iter', 'cost', "matrix_norm"]],
                                                        names=['precision', 'tau', 'var']), dtype='float32')

for eps, tau, i in tqdm(
    itertools.product(eps_to_try, tau_to_try, range(n_batch)), 
    total=len(eps_to_try) * len(tau_to_try) * n_batch):
    # Sample random point clouds 
    X = ContinuousDistribution.random_gaussian_mixture(tau=tau).to_discrete(n_points)
    Y = ContinuousDistribution.random_gaussian_mixture(tau=tau).to_discrete(n_points)
    run_i = entropic_wasserstein_sin(X.support, Y.support, eps)
    result.loc[(eps, tau, ['n_iter', 'cost', "matrix_norm"]), i] = [
        len(run_i.errors), run_i.transport_cost, run_i.cost_matrix_max
    ]
    
result = result.stack().reset_index(name='value')
result['var'] = result['var'].astype('category')
result['tau'] = result['tau'].astype('category')

# Pivot the table
result = result.pivot_table(index=['precision', 'tau', 'run'], columns='var', values='value')
# Reset index with categorical values 
result = result.rename(columns=str).reset_index()
# Makes a new metric
result['ratio_cost_precision'] = result['matrix_norm'] / result['precision']
# log metrics
result['log_ratio'] = np.log10(result['ratio_cost_precision'])
result['log_n_iter'] = np.log10(result['n_iter'])

  0%|          | 0/2000 [00:00<?, ?it/s]/home/gaspardb/miniconda3/envs/OT3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2858: PerformanceWarning:

indexing past lexsort depth may impact performance.

100%|██████████| 2000/2000 [00:10<00:00, 192.99it/s]


In [128]:
fig = px.scatter(result, x='log_ratio', y='log_n_iter', color='tau', trendline='ols',
           title="Number of iterations vs. C/eps",
           labels={'log_ratio': 'log C/eps',
                   'log_n_iter': 'log n_i'}, 
          )
fig.show()

for row in fig._px_trendlines.iterrows():
    # fig._px_trendlines contains a dataframe with the statistical models on each row
    tau, model = float(row[1]['tau']), row[1]['px_fit_results']
    # Slope is in the array of `model.params`; other information available at: 
    # https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html
    print(f"tau={tau:5.1e}. Coeff={model.params[1]:.2f}. R^2={model.rsquared:.2f}. Intercept: {10**(model.params[0]):.1f}")

tau=1.0e-04. Coeff=0.93. R^2=0.82. Intercept: 14.3
tau=1.0e-03. Coeff=0.91. R^2=0.84. Intercept: 12.7
tau=1.0e-02. Coeff=0.98. R^2=0.91. Intercept: 10.1
tau=1.0e-01. Coeff=1.13. R^2=0.97. Intercept: 6.3


### Conclusion

Number of iterations seems to be a $O\left(\coeff\right)$ rather than the best theoretical bound found so far: $O\left(\left[\coeff\right]^2\right)$.

## Is warm starting better? 

### Setup

A given number of $n=100$ points sampled from a GM with covariance $\tau$, and a precision $\epsilon$ are given.
1. First, Sinkhorn is ran over $n-1$ points. The number of iterations $N$ is noted. The dual variables $u, v$ are saved.
2. Then, we add a point $x, y$ in each point cloud. The histograms are updated: $\frac{1}{n-1} \mathbf{1}
\mapsto \frac{1}{n} \mathbf{1}$. A row and a column is added to the cost matrix. Then we run Sinkhorn by initializing the dual variables to $u, v$. The number of iterations of this warm-start Sinkhorn is noted: $N_w$.

The goal is to see whether $N \ll n$ or $N \approx n$.

In [149]:
n_points = 100
n_batch = 100
eps_to_try = np.logspace(0, -.5, 2)
tau_to_try = np.logspace(-1, -4, 2)
warm_to_try = [False, True]
coeff_to_test = np.linspace(.96, 1.04, 1500)


result = pd.DataFrame(columns=['eps', 'tau', 'warm', 'run', 'n_iter', 'cost', "matrix_norm", "coeff"],
                      index=range(len(eps_to_try) * len(tau_to_try) * len(warm_to_try) * n_batch),
                      dtype='float32'
                     )

for k, (eps, tau, i) in tqdm(
    enumerate(itertools.product(eps_to_try, tau_to_try, range(n_batch))), 
    total=len(result)//2):
    
    # Sample random point clouds 
    X = ContinuousDistribution.random_gaussian_mixture(tau=tau).to_discrete(n_points)
    Y = ContinuousDistribution.random_gaussian_mixture(tau=tau).to_discrete(n_points)
    
    # Sinkhorn and warm start
    run_i = entropic_wasserstein_sin(X.support[:-1], Y.support[:-1], eps)
    run_i_warm = entropic_wasserstein_sin_warm(run_i.cost_matrix, X.support[:-1], Y.support[:-1],
                              X.support[-1][:, np.newaxis], Y.support[-1][:, np.newaxis],
                              eps, 
                              run_i.u[:, np.newaxis], run_i.v[:, np.newaxis])
    
    # Min coeff between the dual variables
    coeff = coeff_to_test[np.argmin(np.linalg.norm(run_i.u * coeff_to_test[:, np.newaxis] - run_i_warm.u[:-1], ord=1, axis=1))]
    
    
    result.loc[k] = [
        eps, tau, False, i, len(run_i.errors), run_i.transport_cost, run_i.cost_matrix_max, coeff
    ]
    result.loc[(len(result)//2) + k] = [
        eps, tau, True, i, len(run_i_warm.errors), run_i_warm.transport_cost, run_i_warm.cost_matrix_max, coeff
    ]
    

# # Makes a new metric
result['ratio_cost_eps'] = result['matrix_norm'] / result['eps']
# # log metrics
result['log_ratio'] = np.log10(result['ratio_cost_eps'])
result['log_n_iter'] = np.log10(result['n_iter'])

100%|██████████| 400/400 [00:06<00:00, 62.32it/s] 


In [162]:
fig = px.violin(result, y="n_iter", color="warm", facet_row='tau', facet_col='eps', 
                hover_data=result.columns,
                box=True, height=500,
                )
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True, col=2) # assuming second facet
fig.show()

In [160]:
fig = go.Figure()

values = list(itertools.product(result['eps'].unique(), result['tau'].unique()))
names = list(map(lambda val: f"{val[0]:.2f}, {val[1]:.1e}", values))

for i, (eps, tau) in enumerate(values):
    coeff_eps_tau = result['coeff'][(result['eps']==eps) & (result['tau']==tau)]
    fig.add_trace(go.Violin(x=np.zeros_like(coeff_eps_tau)+i,
                            y=coeff_eps_tau,
                            box_visible=True,
                            meanline_visible=True,
                            points='all',
                            name=names[i]
                            ),
                  )


fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(i+1)),
        ticktext = names
    )
)
fig.show()

### Conclusion

There seems to be no benefit from initializing with the previous dual potentials ! Need a more clever update. More annoying, there does not seem to be a single coefficient to initialize the dual variables with.